In [23]:
%load_ext autoreload
%autoreload 2

In [32]:
import spacy
import pandas as pd
from pprint import pprint
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import plotly.express as px
import re
import plotly.express as px
from spacy import displacy
from spacy.symbols import NOUN, DET, ADJ


In [37]:
nlp  = spacy.load('nl_core_news_lg') 

In [35]:
df = pd.read_pickle('./data/processed/df_wiki_text.pickle')

In [33]:
def pre_process_text_generator(text_list):
    
    cleanr = re.compile('<.*?>')
    for text in text_list:
        if isinstance(text, str):
            text = text.lower()                 # Converting to lowercase

            text = re.sub(cleanr, ' ', text)                 # Removing HTML tags
            text = re.sub(r'[?|!|\'|"|#]',r'',text)
            text = re.sub(r'[.|,|)|(|\|/]',r' ',text)        # Removing Punctuations

            yield text
        


In [36]:
def extract_de_of_het(doc):
    substantieven = []
 
    for token in doc:

    #     print(token.text, token.dep_, token.head.text, token.head.pos_,
    #             [child for child in token.children])


        if token.pos == NOUN:
            # Case 1 : If there is het of de in the POS then extract the head text
    #         print(token.text, token.dep_, token.head.text, token.head.pos_,
    #             [child for child in token.children])


            # Case 2 : There is no clear DET. There must be some ADJ before the noun.

            # if the last letter of the adjective is 'e' and  then DET is always de
            children = [child for child in token.children]
            has_adjective = any([child.pos == ADJ for child in token.children])
            has_det = any([child.pos == DET for child in token.children])
            has_de_of_het = any([child.lemma_ == 'de' or  child.lemma_ == 'het' for child in token.children])
            has_geen_of_een = any([child.lemma_ == 'geen' or  child.lemma_ == 'een' for child in token.children])
            if has_adjective or has_det:
                found_adj_with_last_letter_e=False

                if not has_de_of_het and has_adjective and has_geen_of_een:

                    for child in children:
                        if child.pos == ADJ:
                            if child.text[-1] == 'e':
                                substantieven.append(  
                                    {'det': 'de',
                                     'woord' : token.text,
                                     'woord_vec':token.vector
                                    } )
                                found_adj_with_last_letter_e=True
                                break
                    if not found_adj_with_last_letter_e:

                        substantieven.append( {'det': 'het','woord' : token.text,'woord_vec':token.vector} )
#                     print(f"Token text  {token.text}")     
#                     print(f"Token children  {children}")     
#                     print(f"Found adjective ending with 'e' :{found_adj_with_last_letter_e}" )

                if has_de_of_het and not has_geen_of_een :

                    for child in children:
                        if child.pos == DET:
                            substantieven.append( {'det': child.lemma_,'woord' : token.text,'woord_vec':token.vector} )
                            break

        
                
    
    return substantieven



    #         if token.lemma_[-1] == 'e': 
    #             substantieven.append({'det': 'het','woord' : token.head.lemma_})
    #         else : # Else it is a de woord
    #             substantieven.append({'det': 'de','woord' : token.head.lemma_})


In [32]:
cleaned_text = pre_process_text_generator(df.text.tolist())

substantieven_doc_wise = []
with nlp.disable_pipes(["ner"]):
    docs = nlp.pipe(cleaned_text,n_process=5)
    print(f"Using pipeline : {nlp.pipe_names}")

    for doc in docs:
        lidwoordenlijst = extract_de_of_het(doc)
        substantieven_doc_wise.append(lidwoordenlijst)



Using pipeline : ['tagger', 'parser']


In [33]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [34]:
len(substantieven_doc_wise)

3051

In [46]:
df_processed = pd.DataFrame.from_dict(flatten(substantieven_doc_wise))

In [44]:
df_processed.head()

,det,woord,woord_vec
0,de,gemeente,"[1.0568, -0.94515, 0.99421, -0.68407, 2.9815, ..."
1,de,provincie,"[1.716, -1.4213, -0.50323, -2.4624, 1.5628, 0...."
2,het,dorp,"[1.9532, 0.047137, -0.70945, -2.8567, -0.33987..."
3,het,dorp,"[1.9532, 0.047137, -0.70945, -2.8567, -0.33987..."
4,de,eeuw,"[5.555, -2.933, -2.232, -2.0864, 1.9924, 0.687..."


In [48]:
# Remove duplicates from name
df_processed.drop_duplicates(subset='woord',inplace=True)

In [19]:
df_processed.to_pickle('./data/dumps/woorden_met_hetofde.pickle')
